In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

df=pd.read_csv('alljobs_221117_all.csv')
df.shape

(884, 7)

### Preprocesamiento

In [2]:
# Todo a minusculas
df['jd']=df['Job Details'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Job Title']=df['Job Title'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Company']=df['Company'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Location']=df['Location'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#Removing special char
import re
df['jd1']=df['jd'].map(lambda x: re.sub('<[^<]+?>', '', x))
df['jd1']=df['jd1'].map(lambda x: re.sub(r'\W+',' ',x))
df['jd1']=df['jd1'].map(lambda x: re.sub(r' n ',' ',x))
df['jd1']=df['jd1'].map(lambda x: re.sub(r' n about the job n n ',' ',x))

df['Location']=df['Location'].map(lambda x: re.sub(r'\W+',' ',x))
df['Location']=df['Location'].map(lambda x: re.sub(r'switzerland',' ',x))

df['iwc']=df['jd1'].apply(lambda x: len(str(x).split(" ")))      #Initial Word Count
df['mean']=df['iwc'].mean()
df['inc']=df['jd1'].str.len()                                    #Initial Number of Character
df['num'] = df['jd1'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df = df.reset_index()
df.shape

(884, 14)

## Extract keywords from all job descriptions

<ol>
  <li>business</li>
  <li>team</li>
  <li>management</li>
  <li>support</li>
  <li>development</li>
  <li>analytic solution</li>
  <li>sql - - - - - - - - - - - - - - - - <strong>f_sql</strong></li>
  <li>english - - - - - - - - - - - - - - <strong>f_english</strong></li>
  <li>cloud - - - - - - - - - - - - - - - <strong>f_cloud</strong></li>
  <li>agile - - - - - - - - - - - - - - - <strong>f_agile</strong></li>
  <li>python - - - - - - - - - - - - - - .<strong>f_python</strong></li>
  <li>implementation</li>
  <li>models</li>
  <li>drive</li>
  <li>impact</li>
  <li>insights- - - - - - - - - - - - - - <strong>f_insights</strong></li>
  <li>azure 183 - - - - - - - - - - - - - <strong>f_azure</strong></li>
  <li>german - - - - - - - - - -  - - - - <strong>f_german</strong></li>
  <li>google</li>
  <li>integration - - - - - - - - - - - - <strong>flag</strong></li>
  <li>delivery - - - - - - - -  - - - - - <strong>flag</strong></li>
  <li>statistical - - - - - - - - - - - - <strong>flag</strong></li>
  <li>collaborate - - - - - - - - - - - - <strong>flag</strong></li>
  <li>big 124 - - - - - - - - - - - - - - <strong>flag</strong></li>
  <li>banking 112</li>
  <li>excel 109 - - - - - - - - - - - - - - <strong>flag</strong></li>
  <li>tableau 102 - - - - - - - - - - - - - - <strong>flag</strong></li>
  <li>visualization - - - - - - - - - - - - - - <strong>flag</strong></li>
  <li>dashboards - - - - - - - - - - - - - - <strong>flag</strong></li>
  <li>clinical</li>
  <li>java 89</li>
  <li>pasionate</li>
  <li>mathematics</li>
  <li>pharmaceutical</li>
  <li>sas 73 - - - - - - - - - - - - - - <strong>flag</strong></li>
  <li>pipelines - - - - - - - - - - - - - - <strong>flag</strong></li>
  <li>oracle 63</li>
  <li>aws 58 - - - - - - - - - - - - - - <strong>flag</strong></li>
  <li>warehouse - - - - - - - - - - - - - - <strong>flag</strong></li>
  <li>citizenship</li>
  <li>linux - - - - - - - - - - - - - - <strong>flag</strong></li>
  <li>scrum - - - - - - - - - - - - - - <strong>flag</strong></li>
  <li>pandas - - - - - - - - - - - - - - <strong>flag</strong></li>
</ol>

In [3]:
pd.set_option('display.max_rows', None)
# Palabras frecuentes
freq = pd.Series(' '.join(df['jd1']).split()).value_counts()[:3]
print(freq)

and    14821
the     7732
to      7546
dtype: int64


In [4]:
#Creacion de flags para aleman

df['und']=df['jd1'].apply(lambda x: 1 if ' und ' in x else 0)
df['der']=df['jd1'].apply(lambda x: 1 if ' der ' in x else 0)
df['die']=df['jd1'].apply(lambda x: 1 if ' die ' in x else 0)
df['mit']=df['jd1'].apply(lambda x: 1 if ' mit ' in x else 0)
df['von']=df['jd1'].apply(lambda x: 1 if ' von ' in x else 0)
df['für']=df['jd1'].apply(lambda x: 1 if ' für ' in x else 0)
df['sie']=df['jd1'].apply(lambda x: 1 if ' wir ' in x else 0)
df['sum_de']=df['und']+df['der']+df['die']+df['mit']+df['von']+df['für']+df['sie']
df['flg_DEUTSCH']=df['sum_de'].apply(lambda x: 1 if x > 4 else 0)

In [5]:

df['f_sql'] = df['jd1'].apply(lambda x: 1 if 'sql' in x else 0)
df['f_english'] = df['jd1'].apply(lambda x: 1 if 'englis' in x else 0)
df['f_cloud'] = df['jd1'].apply(lambda x: 1 if 'cloud' in x else 0)
df['f_agile'] = df['jd1'].apply(lambda x: 1 if 'agile' in x else 0)
df['f_python'] = df['jd1'].apply(lambda x: 1 if 'python' in x else 0)
df['f_insights'] = df['jd1'].apply(lambda x: 1 if 'insights' in x else 0)
df['f_azure'] = df['jd1'].apply(lambda x: 1 if 'azure' in x else 0)
df['f_german'] = df['jd1'].apply(lambda x: 1 if 'german' in x else 0)

df['f_r'] = df['jd1'].apply(lambda x: 1 if ' r ' in x else 0)
df['f_java'] = df['jd1'].apply(lambda x: 1 if ' java ' in x else 0)
df['f_javascript'] = df['jd1'].apply(lambda x: 1 if ' javascript ' in x else 0)
df['f_bigdata'] = df['jd1'].apply(lambda x: 1 if 'big data' in x else 0)
df['f_pandas'] = df['jd1'].apply(lambda x: 1 if 'pandas' in x else 0)
df['f_ml'] = df['jd1'].apply(lambda x: 1 if 'machine learning' in x else 0)
df['f_ml1'] = df['jd1'].apply(lambda x: 1 if ' ml ' in x else 0)
df['f_dl'] = df['jd1'].apply(lambda x: 1 if 'deep learning' in x else 0)

df['f_algorithms'] = df['jd1'].apply(lambda x: 1 if 'algorithms' in x else 0)
df['f_engineer'] = df['jd1'].apply(lambda x: 1 if 'engineer' in x else 0)
df['f_scala'] = df['jd1'].apply(lambda x: 1 if ' scala ' in x else 0)
df['f_tableau'] = df['jd1'].apply(lambda x: 1 if 'tableau' in x else 0)
df['f_qlik'] = df['jd1'].apply(lambda x: 1 if 'qlik' in x else 0)
df['f_cloud'] = df['jd1'].apply(lambda x: 1 if 'cloud' in x else 0)
df['f_twitter'] = df['jd1'].apply(lambda x: 1 if 'twitter' in x else 0)

df['f_mathematics'] = df['jd1'].apply(lambda x: 1 if 'mathemati' in x else 0)
df['f_pipelines'] = df['jd1'].apply(lambda x: 1 if 'pipelines' in x else 0)
df['f_zurich'] = df['jd1'].apply(lambda x: 1 if 'zurich' in x else 0)
df['f_geneva'] = df['jd1'].apply(lambda x: 1 if 'geneva' in x else 0)
df['f_clinical'] = df['jd1'].apply(lambda x: 1 if 'clinical' in x else 0)
df['f_salesforce'] = df['jd1'].apply(lambda x: 1 if 'salesforce' in x else 0)

df['f_deutsch'] = df['jd1'].apply(lambda x: 1 if 'deutsch' in x else 0)
df['f_german'] = df['jd1'].apply(lambda x: 1 if 'german' in x else 0)
df['f_google'] = df['jd1'].apply(lambda x: 1 if 'google' in x else 0)
df['f_programming'] = df['jd1'].apply(lambda x: 1 if 'develop' in x else 0)
df['f_entwickeln'] = df['jd1'].apply(lambda x: 1 if 'entwickeln' in x else 0)

df['f_developer'] = df['jd1'].apply(lambda x: 1 if 'programming' in x else 0)
df['f_finance'] = df['jd1'].apply(lambda x: 1 if 'finance' in x else 0)
df['f_fraud'] = df['jd1'].apply(lambda x: 1 if 'fraud' in x else 0)

df['f_credit'] = df['jd1'].apply(lambda x: 1 if 'credit' in x else 0)
df['f_bank'] = df['jd1'].apply(lambda x: 1 if 'bank' in x else 0)
df['f_dashboards'] = df['jd1'].apply(lambda x: 1 if 'dashboard' in x else 0)
df['f_excel'] = df['jd1'].apply(lambda x: 1 if 'powerpoint' in x else 0)
df['f_powerpoint'] = df['jd1'].apply(lambda x: 1 if 'excel' in x else 0)
df['f_agile'] = df['jd1'].apply(lambda x: 1 if 'agile' in x else 0)
df['f_database'] = df['jd1'].apply(lambda x: 1 if 'database' in x else 0)
df['f_statistic'] = df['jd1'].apply(lambda x: 1 if 'statisti' in x else 0)
df['f_visualization'] = df['jd1'].apply(lambda x: 1 if 'visualization' in x else 0)
df['f_bachelor'] = df['jd1'].apply(lambda x: 1 if 'bachelor' in x else 0)
df['f_businessintelligence'] = df['jd1'].apply(lambda x: 1 if 'business intelligence' in x else 0)
df['f_insight'] = df['jd1'].apply(lambda x: 1 if 'insight' in x else 0)
df['f_modeling'] = df['jd1'].apply(lambda x: 1 if 'modeling' in x else 0)
df['f_sas'] = df['jd1'].apply(lambda x: 1 if 'sas' in x else 0)
df['f_hadoop'] = df['jd1'].apply(lambda x: 1 if 'hadoop' in x else 0)
df['f_hive'] = df['jd1'].apply(lambda x: 1 if 'hive' in x else 0)
df['f_aws'] = df['jd1'].apply(lambda x: 1 if 'aws' in x else 0)
df['f_azure'] = df['jd1'].apply(lambda x: 1 if 'azure' in x else 0)
df['f_oracle'] = df['jd1'].apply(lambda x: 1 if 'oracle' in x else 0)

df['f_docker'] = df['jd1'].apply(lambda x: 1 if 'docker' in x else 0)
df['f_kpi'] = df['jd1'].apply(lambda x: 1 if 'kpi' in x else 0)
df['f_agile'] = df['jd1'].apply(lambda x: 1 if 'agile' in x else 0)
df['f_ch'] = df['jd1'].apply(lambda x: 1 if 'switzerland' in x else 0)
#df['f_short'] = df['iwc'].apply(lambda x: 1 if df['iwc']<df['iwc'] else 0)

df['id_job'] = df['idjob'].str[34:60]
df['id_job'] = df['id_job'].str[1:11]

### Salida de datos

In [6]:
salida=df[["Job Title","Company","jd1","Location","idjob","iwc","flg_DEUTSCH","f_python","f_qlik","f_tableau","f_sql",
           "f_agile","f_azure","f_sas","f_visualization","f_statistic","f_database"]]

### Changelog
***

<table>
  <tr>
    <th>Time/Date</th>
    <th>Author</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>18:59 17/11/2022</td>
    <td>Luis Figueroa Castillo</td>
    <td>Obtaining keywords from job description</td>
  </tr>
 
</table>
